In [12]:
import pandas as pd
import numpy as np

In [2]:
missing1_base = pd.read_csv("adult_1group.csv")
missing2_base = pd.read_csv("adult_2groups.csv")

In [3]:
missing1_others = {
    "kmeans (eval on recon)": "adult_1group_kmeans_eval_on_recon.csv",
    "ranking (eval on recon)": "adult_1group_ranking_eval_on_recon.csv",
    "kmeans (supersample)": "adult_1group_kmeans_supersample.csv",
    "ranking (supersample)": "adult_1group_ranking_supersample.csv",
    "E&S (true_balancing)": "adult_1group_ens_balanced_context.csv",
    "E&S (no_balancing)": "adult_1group_ens_no_balancing.csv",
}
missing2_others = {
    "kmeans (eval on recon)": "adult_2groups_kmeans_eval_on_recon.csv",
    "ranking (eval on recon)": "adult_2groups_ranking_eval_on_recon.csv",
    "kmeans (supersample)": "adult_2groups_kmeans_supersample.csv",
    "ranking (supersample)": "adult_2groups_ranking_supersample.csv",
}

def normalize_others(file_dict):
    df_all = pd.DataFrame()
    for approach, filename in file_dict.items():
        df = pd.read_csv(filename)
        df.insert(0, "approach", approach)
        df_all = pd.concat([df_all, df], axis="index", ignore_index=True, sort=False)
    return df_all
missing1_others = normalize_others(missing1_others)
missing2_others = normalize_others(missing2_others)

In [4]:
dro_missing1 = pd.read_csv("adult_dro_1group.csv")
dro_missing2 = pd.read_csv("adult_dro_2group.csv")

# normalizing this data:
def normalize_dro_results(df):
    df.insert(0, "approach", "no_balancing")
    df.insert(2, "data", "original_data")
    df.insert(4, "wandb_url", "(None)")
    df.insert(5, "cluster_test_acc", float("nan"))
    df.insert(6, "cluster_context_acc", float("nan"))
    df.rename(columns={
        "Seed": "seed",
        "eta": "method",
        "prob_pos_sens_0.0/sens_1.0": "prob_pos_sex_Male_0.0/sex_Male_1.0",
        "TPR_sens_0.0/sens_1.0": "TPR_sex_Male_0.0/sex_Male_1.0",
        "TNR_sens_0.0/sens_1.0": "TNR_sex_Male_0.0/sex_Male_1.0",
    }, inplace=True)
    df["method"] = df["method"].transform(lambda eta: f"DRO (eta={eta})")
    
normalize_dro_results(dro_missing1)
normalize_dro_results(dro_missing2)

In [5]:
missing1 = pd.concat([missing1_base, dro_missing1, missing1_others], axis="index", ignore_index=True, sort=False)
missing2 = pd.concat([missing2_base, dro_missing2, missing1_others], axis="index", ignore_index=True, sort=False)

In [19]:
missing1.rename(columns={"data": "type", "method": "classifier"}, inplace=True)
missing1["type"].replace({"x_rand_s": "ours", "original_data": "baseline"}, inplace=True)
missing1[["type", "approach", "classifier", "Accuracy", "prob_pos_sex_Male_0.0/sex_Male_1.0", "TPR_sex_Male_0.0/sex_Male_1.0", "TNR_sex_Male_0.0/sex_Male_1.0", "cluster_context_acc"]].groupby(["type", "approach", "classifier"]).agg(lambda x: f"{round(np.mean(x), 3)} $\\pm$ {round(np.std(x), 3)}").query(
"~(type == 'baseline' & (approach.str.contains('E&S') | approach.str.contains('kmeans') | approach.str.contains('ranking') | approach == 'true_balancing')) & ~(type == 'ours' & classifier == 'LRCV')")

Accuracy  \
type     approach                classifier                                       
baseline no_balancing            DRO (eta=0.01)               0.663 $\pm$ 0.006   
                                 DRO (eta=0.1)                0.663 $\pm$ 0.008   
                                 DRO (eta=0.5)                0.664 $\pm$ 0.009   
                                 DRO (eta=1.0)                    0.5 $\pm$ 0.0   
                                 Kamiran & Calders LR         0.638 $\pm$ 0.008   
                                 LRCV                         0.663 $\pm$ 0.005   
                                 Logistic Regression (C=1.0)  0.664 $\pm$ 0.005   
                                 Majority                         0.5 $\pm$ 0.0   
                                 SVM                          0.657 $\pm$ 0.004   
ours     E&S (no_balancing)      Logistic Regression (C=1.0)  0.677 $\pm$ 0.028   
         E&S (true_balancing)    Logistic Regression (C=1.0)  0.651 $\pm$ 0.036   
         kmeans                  Logistic Regression (C=1.0)  0.671 $\pm$ 0.025   
         kmeans (eval on recon)  Logistic Regression (C=1.0)  0.635 $\pm$ 0.028   
         kmeans (supersample)    Logistic Regression (C=1.0)  0.661 $\pm$ 0.014   
         no_balancing            Logistic Regression (C=1.0)  0.667 $\pm$ 0.015   
         ranking                 Logistic Regression (C=1.0)   0.68 $\pm$ 0.021   
         ranking (eval on recon) Logistic Regression (C=1.0)  0.675 $\pm$ 0.028   
         ranking (supersample)   Logistic Regression (C=1.0)  0.679 $\pm$ 0.025   
         true_balancing          Logistic Regression (C=1.0)  0.656 $\pm$ 0.044   

                                                             prob_pos_sex_Male_0.0/sex_Male_1.0  \
type     approach                classifier                                                       
baseline no_balancing            DRO (eta=0.01)                                0.182 $\pm$ 0.02   
                                 DRO (eta=0.1)                                0.186 $\pm$ 0.027   
                                 DRO (eta=0.5)                                  0.19 $\pm$ 0.04   
                                 DRO (eta=1.0)                                    1.0 $\pm$ 0.0   
                                 Kamiran & Calders LR                         0.223 $\pm$ 0.045   
                                 LRCV                                         0.171 $\pm$ 0.027   
                                 Logistic Regression (C=1.0)                  0.175 $\pm$ 0.027   
                                 Majority                                         nan $\pm$ nan   
                                 SVM                                          0.233 $\pm$ 0.039   
ours     E&S (no_balancing)      Logistic Regression (C=1.0)                   0.525 $\pm$ 0.26   
         E&S (true_balancing)    Logistic Regression (C=1.0)                  0.512 $\pm$ 0.228   
         kmeans                  Logistic Regression (C=1.0)                  0.313 $\pm$ 0.154   
         kmeans (eval on recon)  Logistic Regression (C=1.0)                  0.265 $\pm$ 0.122   
         kmeans (supersample)    Logistic Regression (C=1.0)                  0.234 $\pm$ 0.056   
         no_balancing            Logistic Regression (C=1.0)                  0.278 $\pm$ 0.166   
         ranking                 Logistic Regression (C=1.0)                  0.354 $\pm$ 0.122   
         ranking (eval on recon) Logistic Regression (C=1.0)                  0.571 $\pm$ 0.245   
         ranking (supersample)   Logistic Regression (C=1.0)                   0.34 $\pm$ 0.164   
         true_balancing          Logistic Regression (C=1.0)                  0.457 $\pm$ 0.091   

                                                             TPR_sex_Male_0.0/sex_Male_1.0  \
type     approach                classifier                                                  
baseline no_balancing            DRO (eta=0.01)            

In [8]:
missing2.rename(columns={"data": "type", "method": "classifier"}, inplace=True)
missing2["type"].replace({"x_rand_s": "ours", "original_data": "baseline"}, inplace=True)
missing2.groupby(["type", "approach", "classifier"]).agg(["mean", "std"])[["Accuracy", "prob_pos_sex_Male_0.0/sex_Male_1.0", "TPR_sex_Male_0.0/sex_Male_1.0", "TNR_sex_Male_0.0/sex_Male_1.0", "cluster_context_acc"]].query(
"~(type == 'baseline' & (approach.str.contains('kmeans') | approach.str.contains('ranking') | approach == 'true_balancing')) & ~(type == 'ours' & classifier == 'LRCV')")

Accuracy  \
                                                                  mean   
type     approach                classifier                              
baseline no_balancing            DRO (eta=0.01)               0.776497   
                                 DRO (eta=0.1)                0.774401   
                                 DRO (eta=0.5)                0.770808   
                                 DRO (eta=1.0)                0.500000   
                                 Kamiran & Calders LR         0.747605   
                                 LRCV                         0.740269   
                                 Logistic Regression (C=1.0)  0.747605   
                                 Majority                     0.500000   
                                 SVM                          0.708533   
ours     kmeans                  Logistic Regression (C=1.0)  0.713772   
         kmeans (eval on recon)  Logistic Regression (C=1.0)  0.635479   
         kmeans (supersample)    Logistic Regression (C=1.0)  0.660928   
         no_balancing            Logistic Regression (C=1.0)  0.735329   
         ranking                 Logistic Regression (C=1.0)  0.747156   
         ranking (eval on recon) Logistic Regression (C=1.0)  0.675150   
         ranking (supersample)   Logistic Regression (C=1.0)  0.678743   
         true_balancing          Logistic Regression (C=1.0)  0.745060   

                                                                        \
                                                                   std   
type     approach                classifier                              
baseline no_balancing            DRO (eta=0.01)               0.031126   
                                 DRO (eta=0.1)                0.032306   
                                 DRO (eta=0.5)                0.031559   
                                 DRO (eta=1.0)                0.000000   
                                 Kamiran & Calders LR         0.010090   
                                 LRCV                         0.025706   
                                 Logistic Regression (C=1.0)  0.010090   
                                 Majority                     0.000000   
                                 SVM                          0.004287   
ours     kmeans                  Logistic Regression (C=1.0)  0.071351   
         kmeans (eval on recon)  Logistic Regression (C=1.0)  0.031646   
         kmeans (supersample)    Logistic Regression (C=1.0)  0.015512   
         no_balancing            Logistic Regression (C=1.0)  0.030987   
         ranking                 Logistic Regression (C=1.0)  0.022099   
         ranking (eval on recon) Logistic Regression (C=1.0)  0.031079   
         ranking (supersample)   Logistic Regression (C=1.0)  0.027478   
         true_balancing          Logistic Regression (C=1.0)  0.009022   

                                                             prob_pos_sex_Male_0.0/sex_Male_1.0  \
                                                                                           mean   
type     approach                classifier                                                       
baseline no_balancing            DRO (eta=0.01)                                        0.785255   
                                 DRO (eta=0.1)                                         0.768603   
                                 DRO (eta=0.5)                                         0.767360   
                                 DRO (eta=1.0)                                         1.000000   
                                 Kamiran & Calders LR                                  0.682288   
                                 LRCV                                                  0.632992   
                                 Logistic Regression (C=1.0)                           0.682288   
                                 Majority                                                   NaN   
           